In [1]:
import numpy as np

# Set Up Environment
from kaggle_environments.envs.halite.helpers import *
from kaggle_environments import evaluate, make
env = make("halite", configuration={ "episodeSteps": 400 }, debug=True)

import submission
# import bot_swarm

env.configuration

{'episodeSteps': 400,
 'agentTimeout': 30,
 'actTimeout': 6,
 'runTimeout': 9600,
 'isProduction': False,
 'startingHalite': 24000,
 'size': 21,
 'spawnCost': 500,
 'convertCost': 500,
 'moveCost': 0,
 'collectRate': 0.25,
 'regenRate': 0.02,
 'maxCellHalite': 500,
 'randomSeed': 1619205120}

In [38]:
board_size = 5
environment = make("halite", configuration={"size": board_size, "startingHalite": 1000})
agent_count = 4
environment.reset(agent_count)
state = environment.state[0]

obs = state.observation
config = environment.configuration

board = Board(state.observation, environment.configuration)
print(board)

| 0 | 0 | 0 | 0 | 0 |
| 2 |a0 | 0 |b0 | 2 |
| 1 | 0 | 0 | 0 | 1 |
| 2 |c0 | 0 |d0 | 2 |
| 0 | 0 | 0 | 0 | 0 |



In [39]:
obs

{'player': 0,
 'step': 0,
 'halite': [0,
  0,
  0,
  0,
  0,
  159,
  0,
  0,
  0,
  159,
  61,
  27,
  0,
  27,
  61,
  159,
  0,
  0,
  0,
  159,
  0,
  0,
  0,
  0,
  0],
 'players': [[5000, {}, {'0-1': [6, 0]}],
  [5000, {}, {'0-2': [8, 0]}],
  [5000, {}, {'0-3': [16, 0]}],
  [5000, {}, {'0-4': [18, 0]}]]}

In [24]:
def get_coordinates(pos: int, size: int) -> np.ndarray:
    return np.array([pos % size, pos // size])


def get_position(coordinates: np.ndarray, size: int) -> int:
    return coordinates[0] + coordinates[1] * size

In [56]:
def distance_1d(val_1: Union[int, np.ndarray], val_2: Union[int, np.ndarray], size: int) -> Union[int, np.ndarray]:
    min_val = np.fmin(val_1, val_2)
    max_val = np.fmax(val_1, val_2)
    return np.fmin(max_val - min_val, min_val + size - max_val)


def manhatten_distance(pos_1: int, pos_2: int, size: int) -> int:
    coordinates_1 = get_coordinates(pos_1, size)
    coordinates_2 = get_coordinates(pos_2, size)
    
    dx = distance_1d(coordinates_1[0], coordinates_2[0], size)
    dy = distance_1d(coordinates_1[1], coordinates_2[1], size)
    
    return dx + dy


def create_distance_matrix(size: int) -> np.ndarray:    
    # Maximal position is size**2 - 1
    position_range = np.arange(size**2)
    positions_1 = np.repeat(position_range, size**2)
    positions_2 = np.tile(position_range, size**2)
    
    # positions_1 and positions_2 combined contain all possible combinations of positions
    x_distances = distance_1d(positions_1 % size, positions_2 % size, size)
    y_distances = distance_1d(positions_1 // size, positions_2 // size, size)
    
    distance_matrix = (x_distances + y_distances).reshape(size**2, -1)
    
    return distance_matrix

In [60]:
dist_mat = create_distance_matrix(3)

In [66]:
dist_mat

array([[0, 1, 1, 1, 2, 2, 1, 2, 2],
       [1, 0, 1, 2, 1, 2, 2, 1, 2],
       [1, 1, 0, 2, 2, 1, 2, 2, 1],
       [1, 2, 2, 0, 1, 1, 1, 2, 2],
       [2, 1, 2, 1, 0, 1, 2, 1, 2],
       [2, 2, 1, 1, 1, 0, 2, 2, 1],
       [1, 2, 2, 1, 2, 2, 0, 1, 1],
       [2, 1, 2, 2, 1, 2, 1, 0, 1],
       [2, 2, 1, 2, 2, 1, 1, 1, 0]], dtype=int32)

In [65]:
dist_mat[0, 5]

2

In [55]:
distance_1d(0, 4, 5)

1

In [54]:
distance_1d(np.array([0, 1, 2, 3, 4]), np.array([4, 3, 2, 1, 0]), 5)

array([1, 2, 0, 2, 1])

In [25]:
base = np.arange(5**2)
idx1 = np.repeat(base, 5**2)
idx2 = np.tile(base, 5**2)

In [27]:
idx2

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24,  0,  1,  2,  3,  4,  5,  6,  7,  8,
        9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,  0,
        1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
       10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,  0,  1,
        2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
       11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,  0,  1,  2,
        3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,
       12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,  0,  1,  2,  3,
        4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24,  0

In [18]:
%%time

def dist_1d(a1, a2):
    amin = np.fmin(a1, a2)
    amax = np.fmax(a1, a2)
    adiff = amax-amin
    adist = np.fmin(adiff, 21-adiff)
    return adist


def make_dist_matrix():
    base = np.arange(21**2)
    idx1 = np.repeat(base, 21**2)
    idx2 = np.tile(base, 21**2)

    rowdist = dist_1d(idx1 // 21, idx2 // 21)
    coldist = dist_1d(idx1 % 21, idx2 % 21)

    dist_matrix = (rowdist + coldist).reshape(21**2, -1)
    return dist_matrix

distance_list = make_dist_matrix().tolist()

Wall time: 14 ms


In [4]:
submission.board_halite(obs, config)

array([[115,   0,   0,   0, 115],
       [ 36,   1,   0,   1,  36],
       [ 48,  47,   0,  47,  48],
       [ 36,   1,   0,   1,  36],
       [115,   0,   0,   0, 115]])

In [5]:
def agent(obs, config):
    
    board = Board(obs, config)
    me = board.current_player
    
    print(board)
    
    # Set actions for each ship
    for ship in me.ships:
        ship.next_action = choice([ShipAction.NORTH,ShipAction.EAST,ShipAction.SOUTH,ShipAction.WEST,None])
    
    # Set actions for each shipyard
    for shipyard in me.shipyards:
        shipyard.next_action = None
    
    return me.next_actions

In [14]:
env.run(["submission.py", "bot_swarm.py", "bot_swarm.py", "bot_swarm.py"])
env.render(mode="ipython", width=800, height=600)

KeyboardInterrupt: 